In [2]:
#Imports librairies de base
import numpy as np
import pandas as pd

#Bibliothèques pour Cross validation

#Affichage de Graphes
from IPython.display import Markdown, display, HTML
import matplotlib.pyplot as plt

#Information Mutuelle
from sklearn.feature_selection import mutual_info_regression

In [47]:
# 1.1 Charger les données d'apprentissage (Data/passagers.csv) dans un DataFrame nommé train. Afficher 10 exemples.

train = pd.read_csv("./dev/machineLearning/machineLearning/data/passagers.csv")

train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
721,722,0,3,"Jensen, Mr. Svend Lauritz",male,17.0,1,0,350048,7.0542,NaN,S
567,568,0,3,"Palsson, Mrs. Nils (Alma Cornelia Berglund)",female,29.0,0,4,349909,21.0750,NaN,S
785,786,0,3,"Harmer, Mr. Abraham (David Lishin)",male,25.0,0,0,374887,7.2500,NaN,S
565,566,0,3,"Davies, Mr. Alfred J",male,24.0,2,0,A/4 48871,24.1500,NaN,S
215,216,1,1,"Newell, Miss. Madeleine",female,31.0,1,0,35273,113.2750,D36,C
...,...,...,...,...,...,...,...,...,...,...,...,...
642,643,0,3,"Skoog, Miss. Margit Elizabeth",female,2.0,3,2,347088,27.9000,NaN,S
776,777,0,3,"Tobin, Mr. Roger",male,NaN,0,0,383121,7.7500,F38,Q
566,567,0,3,"Stoytcheff, Mr. Ilia",male,19.0,0,0,349205,7.8958,NaN,S
766,767,0,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,112379,39.6000,NaN,C


In [10]:
# 1.2 Charger les données d'apprentissage (Data/test.csv) dans un Dataframe nommé test. Afficher 10 exemples.

test = pd.read_csv('./dev/machineLearning/machineLearning/data/test.csv')

test.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
15,907,2,"del Carlo, Mrs. Sebastiano (Argenia Genovesi)",female,24.0,1,0,SC/PARIS 2167,27.7208,NaN,C
323,1215,1,"Rowe, Mr. Alfred G",male,33.0,0,0,113790,26.5500,NaN,S
107,999,3,"Ryan, Mr. Edward",male,NaN,0,0,383162,7.7500,NaN,Q
291,1183,3,"Daly, Miss. Margaret Marcella Maggie""""",female,30.0,0,0,382650,6.9500,NaN,Q
65,957,2,"Corey, Mrs. Percy C (Mary Phyllis Elizabeth Mi...",female,NaN,0,0,F.C.C. 13534,21.0000,NaN,S
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
73,965,1,"Ovies y Rodriguez, Mr. Servando",male,28.5,0,0,PC 17562,27.7208,D43,C
132,1024,3,"Lefebre, Mrs. Frank (Frances)",female,NaN,0,4,4133,25.4667,NaN,S
318,1210,3,"Jonsson, Mr. Nils Hilding",male,27.0,0,0,350408,7.8542,NaN,S
350,1242,1,"Greenfield, Mrs. Leo David (Blanche Strouse)",female,45.0,0,1,PC 17759,63.3583,D10 D12,C


In [11]:
# 1.3 Afficher les informations des deux DataFrames train et Test.

train.info()

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [25]:
# 1.4 Calculer le taux global de survie

survived = train['Survived'].mean() * 100

print(round(survived, 2), "de pourcentage de survie.")


38.38 de pourcentage de survie.


In [ ]:
# Question 1 : Quel travail voyez-vous pour que les données soient prêtes à faire tourner dans un modèle ?
# Retravailler les données afin de ne plus afficher des "Nan" dans les colonnes.

# Question 2 : Quelle différence y a-t-il entre les données train et les données test ?
# Il n'y a pas de colonne de survie dans les données test.

# Question 3 : Quel est le rôle de ces deux sets de données ?


In [29]:
# 2.1 Analyser l'effet de la variable Classe sur le taux de survie, en affichant le % de survie par classe

survived_byclass = train.groupby(['Pclass'])['Survived'].mean() * 100

print(survived_byclass)

Pclass
1    62.962963
2    47.282609
3    24.236253
Name: Survived, dtype: float64


In [31]:
# 2.2 Analyser l'effet de la variable Genre (Sex) sur le taux de survie, en affichant le % de survie par genre

survived_bysex = train.groupby(['Sex'])['Survived'].mean() * 100

print(survived_bysex)


Sex
female    74.203822
male      18.890815
Name: Survived, dtype: float64


In [37]:
# 2.3 Calculer une nouvelle colonne FamilySize contenant la taille de la famille, pour les DataFrame train et Test.
# Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et la taille de la famille.

def size_family(row):
    return row['SibSp'] + row['Parch'] + 1

train['FamilySize'] = train.apply(size_family, axis=1)
test['FamilySize'] = test.apply(size_family, axis=1)
print(train[['PassengerId', 'Name', 'Age', 'Sex', 'FamilySize']].sample(10))

     PassengerId                              Name   Age     Sex  FamilySize
26            27           Emir, Mr. Farred Chehab   NaN    male           1
465          466   Goncalves, Mr. Manuel Estanslas  38.0    male           1
875          876  Najib, Miss. Adele Kiamie "Jane"  15.0  female           1
668          669                   Cook, Mr. Jacob  43.0    male           1
527          528                Farthing, Mr. John   NaN    male           1
823          824                Moor, Mrs. (Beila)  27.0  female           2
88            89        Fortune, Miss. Mabel Helen  23.0  female           6
205          206        Strom, Miss. Telma Matilda   2.0  female           2
147          148  Ford, Miss. Robina Maggie "Ruby"   9.0  female           5
331          332               Partner, Mr. Austen  45.5    male           1


In [39]:
# 2.4 Visualisez le nombre de personnes et le taux de survie par taille de famille.

survived_byfamilysize= round(train.groupby(['FamilySize'])['Survived'].mean() * 100, 2)

print(survived_byfamilysize)

FamilySize
1     30.35
2     55.28
3     57.84
4     72.41
5     20.00
6     13.64
7     33.33
8      0.00
11     0.00
Name: Survived, dtype: float64


In [42]:
# 2.5 Calculer une nouvelle colonne IsAlone indiquant si le passager est seul ou pas, pour les DataFrame train et Test. Afficher pour 10 exemples l'Id du passager, le nom, l'age, le sexe et le status IsAlone.

def is_alone(row):
    if row['FamilySize'] == 1 :
        return 1
    else :
        return 0

train['isAlone'] = train.apply(is_alone, axis=1)
test['isAlone'] = test.apply(is_alone, axis=1)

print(train[['PassengerId', 'Name', 'Age', 'Sex', 'isAlone', 'FamilySize']].sample(10))

     PassengerId                                             Name   Age  \
285          286                              Stankovic, Mr. Ivan  33.0   
564          565                   Meanwell, Miss. (Marion Ogden)   NaN   
471          472                                  Cacic, Mr. Luka  38.0   
226          227                        Mellors, Mr. William John  19.0   
121          122                       Moore, Mr. Leonard Charles   NaN   
199          200           Yrois, Miss. Henriette ("Mrs Harbeck")  24.0   
81            82                      Sheerlinck, Mr. Jan Baptist  29.0   
144          145                       Andrew, Mr. Edgardo Samuel  18.0   
385          386                        Davies, Mr. Charles Henry  18.0   
192          193  Andersen-Jensen, Miss. Carla Christine Nielsine  19.0   

        Sex  isAlone  FamilySize  
285    male        1           1  
564  female        1           1  
471    male        1           1  
226    male        1           1  

In [43]:
# 2.6 Visualisez le nombre de personnes et le taux de survie par statut seul ou pas.

survived_aloneOrNot = round(train.groupby(['isAlone'])['Survived'].mean() * 100, 2)

print(survived_aloneOrNot)

isAlone
0    50.56
1    30.35
Name: Survived, dtype: float64


In [ ]:
# 2.7 Calculer l'information mutuelle sur les données train.
# Attention : Il y a des données continues et des données discrètes
# Ignorer les colonnes Name, Ticket et Cabin



In [ ]:
# 2.8 Afficher dans un graphique les scores d'information mutuelle par feature


In [ ]:
# Question 1 : La variable Classe est-elle utile pour prévoir la survie ? Pourquoi ?

# Question 2 : Même question pour la variable Sex

# Question 3 : Comment traiter les variables liées à la famille du passager ?

In [44]:
# 3.1 La variable "Embarked" possède des valeurs nulles, compléter par la valeur majoritaire (S).

train['Embarked'].fillna('S', inplace=True)
test['Embarked'].fillna('S', inplace=True)

print(train[['PassengerId', 'Name', 'Age', 'Sex', 'Embarked']].sample(10))

     PassengerId                                               Name   Age  \
154          155                              Olsen, Mr. Ole Martin   NaN   
666          667                        Butler, Mr. Reginald Fenton  25.0   
338          339                              Dahl, Mr. Karl Edwart  45.0   
774          775              Hocking, Mrs. Elizabeth (Eliza Needs)  54.0   
518          519  Angle, Mrs. William A (Florence "Mary" Agnes H...  36.0   
177          178                         Isham, Miss. Ann Elizabeth  50.0   
144          145                         Andrew, Mr. Edgardo Samuel  18.0   
761          762                     Nirva, Mr. Iisakki Antino Aijo  41.0   
343          344         Sedgwick, Mr. Charles Frederick Waddington  25.0   
279          280                   Abbott, Mrs. Stanton (Rosa Hunt)  35.0   

        Sex Embarked  
154    male        S  
666    male        S  
338    male        S  
774  female        S  
518  female        S  
177  female   

In [45]:
# 3.2 Afficher l'influence du port d'embarquement sur la Survie

survived_embarked = round(train.groupby(['Embarked'])['Survived'].mean() * 100, 2)

print(survived_embarked)

Embarked
C    55.36
Q    38.96
S    33.70
Name: Survived, dtype: float64


In [ ]:
# 3.3 La variable "Fare" (prix du ticket) possède des valeurs manquantes. Remplacer ces valeurs par la valeur médiane.

train['Fare'].fillna(train['Fare'].median(), inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

print(train[['PassengerId', 'Name', 'Age', 'Sex', 'Fare']].sample(10))

In [ ]:
# 3.4 Créer une variable CategoricalFare Découpant la valeur de ticket en 4 catégories de tailles à peu près égales.



In [ ]:
# 3.5 Analyser l'effet de ces catégories de prix de ticket sur la survie.

In [ ]:
# 3.6 Combien de valeurs d'Age manquent ? Complète par des valeurs entières aléatoires entre (moyenne - écart type) et (moyenne + écart type).


In [ ]:
# 3.7 Créer une variable CategoricalAge Découpant la valeur d'age en 5 catégories égales.


In [ ]:
# 3.8 Analyser l'effet de ces catégories d'age sur la survie.

In [ ]:
# Question 1 : Classer les variables par ordre d'intérêt

In [ ]:
# 4.1 : Le nom permet de retrouver le titre du passager. Trouver les titres lorsqu'ils existent.

In [ ]:
# 4.2 : En utilisant la fonction pandas crosstab, afficher pour train le nombre d'occurrences par titres (en ligne) et par sexe (en colonne)


In [ ]:
#4.3 : Remplacer 'Melle et 'Ms' par 'Miss'
#      Remplacer 'Mme' par 'Mrs'
# Créer une valeur "Rare" pour les autres titres avec moins de 10 occurrences.
# Calculer ensuite le taux de survie par titre


In [ ]:
# 5.2 : Attribuer une valeur pour les possibilités de la variable "Title" ('Master','Miss','Mr','Mrs','Rare'). (Question optionnel 4 nécessaires)

In [ ]:
# 5.3 Remplacer les titres manquant par 0, et attribuer à la colonne le type entier. (Question optionnelle 4 nécessaires)


In [ ]:
# 5.4 Attribuer une valeur numérique aux 4 catégories de la variable Fare (question 3.6)

In [ ]:
# 5.5 Attribuer une valeur numérique au Port d'embarquement (Embarked)

In [ ]:
# 5.6 Supprimer les features inutiles


In [ ]:
# 5.7 Afficher les infos pour les deux datasets


In [ ]:
# 5.8 Sauver les deux datasets dans des fichiers csv. Éviter d'écrire un index (n° de ligne)


In [ ]:
# Question : Sur combien de questions avez-vous utilisé les conseils ?
